# Create REVIEWS Search

In [1]:
import random
import json

# Define the total number of individual numbers and the number of buckets
total_numbers = 8750
num_buckets = 2000

# Generate a list of sequential integers from 0 to total_numbers - 1
numbers = list(range(total_numbers))

# Initialize an empty list for each bucket
customers_buckets = [[] for _ in range(num_buckets)]

# Randomly distribute numbers into buckets
for number in random.sample(numbers, len(numbers)):
    # Choose a random bucket and append the number to it
    random_bucket = random.choice(customers_buckets)
    random_bucket.append(number)

In [2]:
with open("customer_list.json", "r") as f:
    customer_list = f.readlines()
    customer_list = [json.loads(i) for i in customer_list]

for i in range(len(customer_list)):
    customer_list[i]["customer_id"] = i

with open("product_reviews.jsonl", "r") as f:
    reviews = f.readlines()
    reviews = [json.loads(i) for i in reviews]

with open("agent_list.json", "r") as f:
    agents = f.readlines()
    agents = [json.loads(i) for i in agents]

In [3]:
# Assign customers to reviews
for customer_id, bucket in enumerate(customers_buckets):
    for element in bucket:
        reviews[element]["customer_id"] = customer_list[customer_id]["customer_id"]
        reviews[element]["customer_email"] = customer_list[customer_id]["email"]

In [5]:
reviews[0]

{'id': '1000',
 'title': "Women's White Cotton Robe - Soft, Absorbent, and Comfortable",
 'description': "Our women's white cotton robe is the perfect way to relax and unwind after a long day. Made from 100% cotton, this robe is soft, absorbent, and comfortable. It features a shawl collar, two patch pockets, and a self-tie belt. This robe is available in sizes S-XXL.\n\nHere are some of the benefits of our women's white cotton robe:\n\n* Made from 100% cotton for softness and absorbency\n* Shawl collar for added warmth\n* Two patch pockets for storing essentials\n* Self-tie belt for a secure fit\n* Available in sizes S-XXL\n\nOrder your women's white cotton robe today and experience the ultimate in comfort and relaxation!",
 'category': 'Bath Robe',
 'review': " I was really looking forward to receiving this robe, but unfortunately, I was disappointed. The material is not as soft as I expected and it's not very absorbent. The stitching is also poor quality and the robe started coming a

In [6]:
for i, review in enumerate(reviews):
    reviews[i]["product_id"] = review["id"]
    del reviews[i]["id"]

    reviews[i]["rating"] = str(reviews[i]["stars"])
    del reviews[i]["stars"]

    reviews[i]["customer_id"] = str(reviews[i]["customer_id"])

In [7]:
reviews[0]

{'title': "Women's White Cotton Robe - Soft, Absorbent, and Comfortable",
 'description': "Our women's white cotton robe is the perfect way to relax and unwind after a long day. Made from 100% cotton, this robe is soft, absorbent, and comfortable. It features a shawl collar, two patch pockets, and a self-tie belt. This robe is available in sizes S-XXL.\n\nHere are some of the benefits of our women's white cotton robe:\n\n* Made from 100% cotton for softness and absorbency\n* Shawl collar for added warmth\n* Two patch pockets for storing essentials\n* Self-tie belt for a secure fit\n* Available in sizes S-XXL\n\nOrder your women's white cotton robe today and experience the ultimate in comfort and relaxation!",
 'category': 'Bath Robe',
 'review': " I was really looking forward to receiving this robe, but unfortunately, I was disappointed. The material is not as soft as I expected and it's not very absorbent. The stitching is also poor quality and the robe started coming apart after just

In [9]:
with open("vertexai_search_reviews.jsonl", "w") as f:
    for i, review in enumerate(reviews):
        f.write(json.dumps(
                {
                    "id":str(i),
                    "jsonData": json.dumps(review),
                    "content": {
                        "mimeType": "text/html", 
                        "uri": f"gs://csm-dataset/p5-dataset/html_reviews/{i}.html"
                    }
                }
            ) + "\n"
        )

In [11]:
html_template = """<html><head><meta http-equiv="Content-Type" content="text/html; charset=windows-1252"></head><body><div class="row mt-5">
	<nav aria-label="breadcrumb">
		<ol class="breadcrumb">
			<li class="breadcrumb-item">Product Category</li>
			<li class="breadcrumb-item active" aria-current="page">{}</li>
		</ol>
	</nav>
	<div class="card mb-12">
		<div class="row g-0">
			<div class="col-md-8">
				<div class="card-body">
					<h3 class="card-title">{}</h3>
                    <p class="card-text"><small>Review</small></p>
                    <p class="card-text"><small>{}</small></p>
                    <p class="card-text"><small>Sentiment: {}</small></p>
					<p class="card-text"><small>Rating: {}</small></p>
				</div>
			</div>
		</div>
	</div>
</div>
</body></html>"""

In [12]:
for i, review in enumerate(reviews):
    with open(f"./dataset/{i}.html", "w") as f:
        content = html_template.format(
            review["category"],
            review["title"],
            review["review"],
            review["sentiment"],
            review["rating"]
        )
        f.write(content)